In [2]:
from datetime import datetime
from pathlib import Path
import json

FECHA = datetime.now().strftime("%Y-%m-%d")
LOG_PATH = Path(f"D:/trading/logs/staging/nb_pipeline_heuristico_{FECHA}.log")
LOG_PATH.parent.mkdir(parents=True, exist_ok=True)

def log_event(modulo, status, mensaje, inicio):
    fin = datetime.now()
    duracion = round((fin - inicio).total_seconds(), 2)
    ts = fin.strftime("%Y-%m-%d %H:%M:%S")
    linea = f"{ts},{modulo},{status},{mensaje},{duracion}s\n"
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(linea)
    print(f"[{modulo}] {status}: {mensaje} ({duracion}s)")

# Cargar simbolos esperados
with open("D:/trading/config/symbol_groups.json", "r") as f:
    grupos = json.load(f)
SIMBOLOS = sorted(set(sum(grupos.values(), [])))
print(f"Simbolos esperados: {len(SIMBOLOS)}")

Simbolos esperados: 50


In [2]:
import subprocess
inicio = datetime.now()
cmd = ["D:/trading_env/Scripts/python.exe", "D:/trading/scripts/core/ingest.py"]
res = subprocess.run(cmd, capture_output=True, text=True)

print("STDOUT:")
print(res.stdout)
print("STDERR:")
print(res.stderr)

status = "OK" if res.returncode == 0 else "ERROR"
log_event("ingest", status, "Proceso terminado", inicio)


inicio = datetime.now()
cmd = ["D:/trading_env/Scripts/python.exe", "D:/trading/scripts/core/ingest.py"]
res = subprocess.run(cmd, capture_output=True, text=True)
log_event("ingest", "OK" if res.returncode == 0 else "ERROR", "Ingesta completada", inicio)
print(res.stdout)

STDOUT:
[ingest] OK: AAPL (5000 filas) (0.24s)
[ingest] OK: AEP (5000 filas) (0.24s)
[ingest] OK: AGNCO (1423 filas) (0.62s)
[ingest] OK: AGNCP (1334 filas) (0.12s)
[ingest] OK: AMD (5000 filas) (0.2s)
[ingest] OK: AMZN (5000 filas) (0.2s)
[ingest] OK: AVGO (3976 filas) (0.18s)
[ingest] OK: CCI (5000 filas) (0.22s)
[ingest] OK: CRBG (676 filas) (0.08s)
[ingest] OK: CRM (5000 filas) (0.2s)
[ingest] OK: CTVA (1510 filas) (0.12s)
[ingest] OK: DGX (5000 filas) (0.21s)
[ingest] OK: DUK (5000 filas) (0.22s)
[ingest] OK: ECCF (341 filas) (0.38s)
[ingest] OK: ED (5000 filas) (0.23s)
[ingest] OK: EPD (5000 filas) (0.22s)
[ingest] OK: EQR (5000 filas) (0.23s)
[ingest] OK: EW (5000 filas) (0.21s)
[ingest] OK: F (5000 filas) (0.22s)
[ingest] OK: FITBO (1433 filas) (0.55s)
[ingest] OK: FWONA (3113 filas) (0.15s)
[ingest] OK: FWONK (2739 filas) (0.15s)
[ingest] OK: GOOGL (5000 filas) (0.28s)
[ingest] OK: HBANL (563 filas) (0.34s)
[ingest] OK: HBANM (995 filas) (0.6s)
[ingest] OK: HBANP (1084 filas) 

In [3]:
from pathlib import Path
import pandas as pd
inicio = datetime.now()
errores = []
for simbolo in SIMBOLOS:
    try:
        df = pd.read_parquet(f"D:/trading/data/historic/{simbolo}.parquet")
        if df.index.name == 'datetime':
            df = df.reset_index()
        df['fecha'] = pd.to_datetime(df['datetime'])
        df['retorno'] = df['close'].pct_change()
        df['ma_5'] = df['close'].rolling(5).mean()
        df['ma_20'] = df['close'].rolling(20).mean()
        df['ema_5'] = df['close'].ewm(span=5).mean()
        df['ema_20'] = df['close'].ewm(span=20).mean()
        df['ma_ratio_5_20'] = df['ma_5'] / df['ma_20']
        df['slope_ma_5'] = df['ma_5'].diff()
        df['slope_ma_5_std'] = df['slope_ma_5'] / df['ma_5'].rolling(5).std()
        df['retorno_5d'] = df['close'].pct_change(5)
        df['momentum_10d'] = df['close'] - df['close'].shift(10)
        df['roc_10d'] = df['close'].pct_change(10)
        df['volatilidad_5d'] = df['retorno'].rolling(5).std()
        df['volatilidad_20d'] = df['retorno'].rolling(20).std()
        df['rango'] = df['high'] - df['low']
        df['dia_semana'] = df['fecha'].dt.dayofweek
        df['es_fin_de_mes'] = df['fecha'].dt.is_month_end.astype(int)
        df.to_parquet(f"D:/trading/data/features/{simbolo}_features.parquet", index=False)
    except Exception as e:
        errores.append(simbolo)
log_event("features", "OK" if not errores else "WARNING", f"{len(SIMBOLOS)-len(errores)} procesados", inicio)

[features] WARNING: 0 procesados (1.13s)


In [4]:
inicio = datetime.now()
cmd = ["D:/trading_env/Scripts/python.exe", "D:/trading/scripts/utils/shp.py"]
res = subprocess.run(cmd, capture_output=True, text=True)
log_event("senales", "OK" if res.returncode == 0 else "ERROR", "Señales heuristicas generadas", inicio)
print(res.stdout)

[senales] OK: Señales heuristicas generadas (7.4s)
Simbolos a procesar: 46
Iniciando procesamiento serial...















































Senales guardadas en: D:\trading\reports\senales_heuristicas\senales_heuristicas_2025-05-28.csv
Total senales: 127261

Duracion total: 6.38 segundos



In [5]:
inicio = datetime.now()
cmd = ["D:/trading_env/Scripts/python.exe", "D:/trading/scripts/core/run_backtest_heuristicoV2.py", "--tp", "0.04", "--sl", "0.02"]
res = subprocess.run(cmd, capture_output=True, text=True)
log_event("backtest", "OK" if res.returncode == 0 else "ERROR", "Backtest heuristico TP4 SL2", inicio)
print(res.stdout)

[backtest] OK: Backtest heuristico TP4 SL2 (8.75s)
Leyendo archivo de simbolos desde: D:\trading\config\json\symbol_groups.json
Backtest guardado en: D:\trading\reports\backtesting\bt_heuristicas_tp4_sl2_2025-05-28.csv
Total operaciones: 0
Duracion total: 7.09 segundos



In [6]:
from glob import glob
archivos = glob("D:/trading/reports/backtesting/resumen_metricas_tp*_*.csv")
print("Archivos resumen generados:")
for a in archivos:
    print("-", Path(a).name)
print("Pipeline finalizado.")

Archivos resumen generados:
- resumen_metricas_tp4_sl2_2025-05-28.csv
- resumen_metricas_tp4_sl3_2025-05-28.csv
- resumen_metricas_tp6_sl2_2025-05-28.csv
- resumen_metricas_tp6_sl3_2025-05-28.csv
Pipeline finalizado.
